In [455]:
import pandas as pd
import numpy as np

In [533]:
data = pd.read_csv('../train_wo_missing_values.csv')
test = pd.read_csv('../test.csv')

In [534]:
data.drop(index=[523,1298], inplace=True)

In [535]:
from transform import transform_dataset

In [536]:
x, y = transform_dataset(data)

In [537]:
x.to_csv('../train_transf.csv', index=False)
y.to_csv('../test_transf.csv', index=False, header=False)

In [538]:
rand_variable = np.random.rand(x.shape[0])*100

In [539]:
x = x.reset_index(drop=True)

In [540]:
x = pd.concat([x, pd.Series(rand_variable, name='Rand_Var')], axis=1)

In [541]:
pca_var = pd.read_csv('../x12d.csv', prefix='pca')

In [542]:
pca_var.shape

(1458, 12)

In [543]:
x = pd.concat([x, pca_var], axis=1)

In [544]:
x.head()

,LotFrontage,LotArea,YearBuilt,MasVnrArea,TotalBsmtSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,...,2,3,4,5,6,7,8,9,10,11
0,13.754179,9.041922,2003,196.0,856,1710,2003.0,211.355516,0,61,...,0.489574,-0.070740,0.080191,-0.396822,0.819830,1.454054,-0.530810,-0.314133,0.675321,-0.309865
1,15.446997,9.169518,1976,0.0,1262,1262,1976.0,183.057985,298,0,...,-0.752644,0.198471,-0.047932,0.915364,-0.400408,-0.380815,-0.143054,0.270766,-1.185431,0.028497
2,14.107516,9.328123,2001,162.0,920,1786,2001.0,230.170899,0,42,...,0.333702,0.316951,0.334385,1.192629,0.223092,0.657625,-0.554016,0.348407,-0.566953,-0.760743
3,13.146282,9.164296,1915,0.0,756,1717,1998.0,240.681574,0,35,...,-0.382669,-1.279727,3.367146,-1.145491,-0.414045,-0.896113,-0.067543,0.869701,0.666970,-1.126284
4,15.870407,9.565214,2000,350.0,1145,2198,2000.0,298.887194,192,84,...,0.377549,0.281931,0.375355,1.157149,0.335261,0.444143,-0.367786,-0.198315,-0.038292,-0.898506


In [519]:
from sklearn import ensemble
randomForest = ensemble.RandomForestRegressor()

In [520]:
randomForest.set_params(n_estimators=100, oob_score=True)
randomForest.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [521]:
#param_test = {'max_features':range(7,20,2),
#              'min_samples_split':range(80,301,20), 'min_samples_leaf':range(5,66,10),
#              'max_depth':range(5,16,2), 'min_samples_split':range(20,101,20),
#              'n_estimators':range(20,1001,10)}
#gsearch = GridSearchCV(estimator = randomForest,
#param_grid = param_test, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
#gsearch.fit(x, y)
#gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

In [522]:
randomForest.score(x, y)

0.9813704358061975

In [523]:
randomForest.oob_score_

0.8630521099837666

In [525]:
feature_importance = list(zip(x.columns, randomForest.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort[:15]

array([(b'ExterQualBinary', 0.32385795), (b'GrLivArea', 0.24573852),
       (b'TotalBsmtSF', 0.08474739), (b'GarageArea', 0.0775832 ),
       (b'NeighborhoodPoor', 0.03696422), (b'YearBuilt', 0.03234286),
       (b'LotArea', 0.02158274), (b'KitchenQualBinary1', 0.01880624),
       (b'CentralAirBinary', 0.01875944),
       (b'OverallQualBinary', 0.01610338), (b'GarageYrBlt', 0.01220436),
       (b'MSSubClassBinary1', 0.00974318),
       (b'OverallCondBinary', 0.00893713), (b'LotFrontage', 0.00882944),
       (b'Rand_Var', 0.0081561 )],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [526]:
[i for (i, j) in feature_sort[0:15]]

[b'ExterQualBinary',
 b'GrLivArea',
 b'TotalBsmtSF',
 b'GarageArea',
 b'NeighborhoodPoor',
 b'YearBuilt',
 b'LotArea',
 b'KitchenQualBinary1',
 b'CentralAirBinary',
 b'OverallQualBinary',
 b'GarageYrBlt',
 b'MSSubClassBinary1',
 b'OverallCondBinary',
 b'LotFrontage',
 b'Rand_Var']

In [97]:
#data = pd.read_csv('../train_wo_missing_values.csv')
#x_, y_ = data.loc[:, data.columns != 'SalePrice'], data.loc[:,'SalePrice']

In [98]:
#categorical_columns = ['MSSubClass','MSZoning','Street','LotShape','LandContour','LotConfig',
#                         'LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
#                         'OverallQual','OverallCond','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
#                         'MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtExposure',
#                         'BsmtFinType1','Heating','HeatingQC','CentralAir','Electrical','KitchenQual',
#                         'Functional','FireplaceQu','GarageType','GarageFinish','PavedDrive',
#                         'SaleType','SaleCondition', 'Utilities']

In [99]:
#x_[categorical_columns] = x_[categorical_columns].astype(str)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [100]:
#dummies = pd.get_dummies(x_.loc[:,categorical_columns], drop_first=True)

In [101]:
#dummies.head()

,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Utilities_NoSeWa
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [102]:
#x_ = pd.concat([x_.drop(columns=categorical_columns), dummies, pd.Series(rand_variable, name='Rand_Var')], axis=1)

In [273]:
import xgboost as xgb

In [274]:
from sklearn.preprocessing import StandardScaler

In [527]:
xgboost = xgb.XGBRegressor()
xgboost.fit(x, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [528]:
xgboost.score(x, y)

0.9320365148125098

In [529]:
feature_importance_xgboost = list(zip(x.columns, xgboost.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_xgboost = np.array(feature_importance_xgboost, dtype=dtype)
feature_sort_xgboost = np.sort(feature_importance_xgboost, order='importance')[::-1]

In [530]:
feature_sort_xgboost[0:10]

array([(b'GrLivArea', 0.16521738), (b'LotArea', 0.10956522),
       (b'TotalBsmtSF', 0.09043478), (b'Rand_Var', 0.0626087 ),
       (b'GarageArea', 0.05913043), (b'GarageYrBlt', 0.05217391),
       (b'YearBuilt', 0.03652174), (b'OverallCondBinary', 0.03478261),
       (b'OpenPorchSF', 0.03478261), (b'WoodDeckSF', 0.02956522)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [531]:
[i for (i, j) in feature_sort_xgboost[0:10]]

[b'GrLivArea',
 b'LotArea',
 b'TotalBsmtSF',
 b'Rand_Var',
 b'GarageArea',
 b'GarageYrBlt',
 b'YearBuilt',
 b'OverallCondBinary',
 b'OpenPorchSF',
 b'WoodDeckSF']

In [545]:
from sklearn.ensemble import GradientBoostingRegressor

In [547]:
gbm = GradientBoostingRegressor(n_estimators=10000, max_features=int(np.sqrt(x.shape[1])), subsample=0.8, learning_rate=0.001)
gbm.fit(x, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=3, max_features=8,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10000, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False)

In [296]:
feature_importance_gbm = list(zip(x.columns, gbm.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]

In [297]:
feature_sort_gbm[:10]

array([(b'GrLivArea', 0.17578132), (b'TotalBsmtSF', 0.10138314),
       (b'LotArea', 0.0771276 ), (b'GarageArea', 0.06339343),
       (b'GarageYrBlt', 0.04692268), (b'YearBuilt', 0.04231704),
       (b'OpenPorchSF', 0.03931287), (b'OverallCondBinary', 0.03324023),
       (b'Rand_Var', 0.03126577), (b'LotFrontage', 0.02498936)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [298]:
[i for (i, j) in feature_sort_gbm[0:10]]

[b'GrLivArea',
 b'TotalBsmtSF',
 b'LotArea',
 b'GarageArea',
 b'GarageYrBlt',
 b'YearBuilt',
 b'OpenPorchSF',
 b'OverallCondBinary',
 b'Rand_Var',
 b'LotFrontage']

In [549]:
gbm = GradientBoostingRegressor(n_estimators=10000, max_features=int(np.sqrt(x.shape[1])), subsample=0.8, learning_rate=0.1, min_samples_split=20, min_samples_leaf=10, max_depth=6)
gbm.fit(x, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=8,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=10,
             min_samples_split=20, min_weight_fraction_leaf=0.0,
             n_estimators=10000, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False)

In [550]:
gbm.score(x, y)

0.9999994576793632

In [300]:
param_test1 = {'n_estimators':range(20,1001,10)}

In [302]:
gsearch1 = GridSearchCV(gbm, param_grid=param_test1, scoring='neg_mean_squared_error', n_jobs=-1, iid=False, cv=10)
gsearch1.fit(z, y)

KeyboardInterrupt: 

In [225]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.02693, std: 0.00619, params: {'n_estimators': 20},
  mean: -0.02224, std: 0.00559, params: {'n_estimators': 30},
  mean: -0.02078, std: 0.00490, params: {'n_estimators': 40},
  mean: -0.01994, std: 0.00447, params: {'n_estimators': 50},
  mean: -0.01991, std: 0.00457, params: {'n_estimators': 60},
  mean: -0.01990, std: 0.00449, params: {'n_estimators': 70},
  mean: -0.01886, std: 0.00435, params: {'n_estimators': 80},
  mean: -0.01945, std: 0.00510, params: {'n_estimators': 90},
  mean: -0.01943, std: 0.00384, params: {'n_estimators': 100},
  mean: -0.01889, std: 0.00461, params: {'n_estimators': 110},
  mean: -0.01920, std: 0.00410, params: {'n_estimators': 120},
  mean: -0.01931, std: 0.00420, params: {'n_estimators': 130},
  mean: -0.01979, std: 0.00382, params: {'n_estimators': 140},
  mean: -0.01870, std: 0.00374, params: {'n_estimators': 150},
  mean: -0.01851, std: 0.00391, params: {'n_estimators': 160},
  mean: -0.01851, std: 0.00390, params: {'n_estimators': 170},


In [211]:
gsearch1.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=6, max_features=7,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=10,
             min_samples_split=20, min_weight_fraction_leaf=0.0,
             n_estimators=970, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False)

In [228]:
feature_importance_gbm = list(zip(x.columns, gsearch1.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.09937365), (b'TotalBsmtSF', 0.08958185),
       (b'LotArea', 0.08888454), (b'Rand_Var', 0.07703054),
       (b'GarageArea', 0.07124028), (b'YearBuilt', 0.06173909),
       (b'LotFrontage', 0.06148287), (b'GarageYrBlt', 0.05483404),
       (b'OpenPorchSF', 0.05390781), (b'MasVnrArea', 0.03995123),
       (b'WoodDeckSF', 0.03563799), (b'ExterQualBinary', 0.01200575),
       (b'OverallCondBinary', 0.01188319),
       (b'OverallQualBinary', 0.01179675),
       (b'SaleConditionBinary2', 0.01159049)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [212]:
gsearch1.best_estimator_.score(x, y)

0.9623001547493379

In [230]:
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(20,101,20)}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=270, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch2.fit(x,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01882, std: 0.00432, params: {'max_depth': 5, 'min_samples_split': 20},
  mean: -0.01855, std: 0.00552, params: {'max_depth': 5, 'min_samples_split': 40},
  mean: -0.01881, std: 0.00446, params: {'max_depth': 5, 'min_samples_split': 60},
  mean: -0.01877, std: 0.00456, params: {'max_depth': 5, 'min_samples_split': 80},
  mean: -0.01842, std: 0.00456, params: {'max_depth': 5, 'min_samples_split': 100},
  mean: -0.01906, std: 0.00398, params: {'max_depth': 7, 'min_samples_split': 20},
  mean: -0.01902, std: 0.00370, params: {'max_depth': 7, 'min_samples_split': 40},
  mean: -0.01849, std: 0.00366, params: {'max_depth': 7, 'min_samples_split': 60},
  mean: -0.01888, std: 0.00513, params: {'max_depth': 7, 'min_samples_split': 80},
  mean: -0.01878, std: 0.00401, params: {'max_depth': 7, 'min_samples_split': 100},
  mean: -0.01939, std: 0.00365, params: {'max_depth': 9, 'min_samples_split': 20},
  mean: -0.01912, std: 0.00425, params: {'max_depth': 9, 'min_samples_split': 40},
  

In [233]:
gsearch2.best_estimator_.score(x, y)

0.9659521021026289

In [231]:
feature_importance_gbm = list(zip(x.columns, gsearch2.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.1395117 ), (b'TotalBsmtSF', 0.10272045),
       (b'LotArea', 0.0754619 ), (b'GarageArea', 0.06993353),
       (b'GarageYrBlt', 0.06962887), (b'YearBuilt', 0.05905611),
       (b'Rand_Var', 0.0559587 ), (b'OpenPorchSF', 0.04666464),
       (b'LotFrontage', 0.04657094), (b'MasVnrArea', 0.0391836 ),
       (b'WoodDeckSF', 0.02927102), (b'ExterCondBinary', 0.02321556),
       (b'CentralAirBinary', 0.01915601),
       (b'OverallCondBinary', 0.01526515),
       (b'SaleConditionBinary2', 0.01247549)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [237]:
param_test3 = {'min_samples_split':range(80,301,20), 'min_samples_leaf':range(5,66,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=270,max_depth=5,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='v',n_jobs=4,iid=False, cv=10)
gsearch3.fit(x, y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01877, std: 0.00398, params: {'min_samples_leaf': 5, 'min_samples_split': 80},
  mean: -0.01821, std: 0.00391, params: {'min_samples_leaf': 5, 'min_samples_split': 100},
  mean: -0.01900, std: 0.00409, params: {'min_samples_leaf': 5, 'min_samples_split': 120},
  mean: -0.01895, std: 0.00427, params: {'min_samples_leaf': 5, 'min_samples_split': 140},
  mean: -0.01884, std: 0.00425, params: {'min_samples_leaf': 5, 'min_samples_split': 160},
  mean: -0.01860, std: 0.00443, params: {'min_samples_leaf': 5, 'min_samples_split': 180},
  mean: -0.01883, std: 0.00435, params: {'min_samples_leaf': 5, 'min_samples_split': 200},
  mean: -0.01956, std: 0.00473, params: {'min_samples_leaf': 5, 'min_samples_split': 220},
  mean: -0.01900, std: 0.00454, params: {'min_samples_leaf': 5, 'min_samples_split': 240},
  mean: -0.01934, std: 0.00464, params: {'min_samples_leaf': 5, 'min_samples_split': 260},
  mean: -0.01908, std: 0.00474, params: {'min_samples_leaf': 5, 'min_samples_split': 280},


In [241]:
gsearch3.best_estimator_.score(x, y)

0.9611365013315731

In [239]:
feature_importance_gbm = list(zip(x.columns, gsearch3.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.13121384), (b'TotalBsmtSF', 0.1114015 ),
       (b'GarageArea', 0.07164409), (b'LotArea', 0.07092367),
       (b'YearBuilt', 0.06058123), (b'GarageYrBlt', 0.05786832),
       (b'OpenPorchSF', 0.05477668), (b'LotFrontage', 0.05353149),
       (b'Rand_Var', 0.04832145), (b'MasVnrArea', 0.04229633),
       (b'WoodDeckSF', 0.02922249), (b'ExterCondBinary', 0.01939933),
       (b'CentralAirBinary', 0.01918167),
       (b'OverallCondBinary', 0.01714515),
       (b'SaleConditionBinary2', 0.01653209)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [240]:
param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=270,max_depth=5, min_samples_split=100, min_samples_leaf=5, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch4.fit(x, y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01821, std: 0.00391, params: {'max_features': 7},
  mean: -0.01961, std: 0.00426, params: {'max_features': 9},
  mean: -0.01936, std: 0.00411, params: {'max_features': 11},
  mean: -0.01915, std: 0.00437, params: {'max_features': 13},
  mean: -0.01981, std: 0.00434, params: {'max_features': 15},
  mean: -0.01949, std: 0.00465, params: {'max_features': 17},
  mean: -0.01959, std: 0.00388, params: {'max_features': 19}],
 {'max_features': 7},
 -0.018214793335008455)

In [242]:
gsearch4.best_estimator_.score(x, y)

0.9611365013315731

In [243]:
feature_importance_gbm = list(zip(x.columns, gsearch4.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.13121384), (b'TotalBsmtSF', 0.1114015 ),
       (b'GarageArea', 0.07164409), (b'LotArea', 0.07092367),
       (b'YearBuilt', 0.06058123), (b'GarageYrBlt', 0.05786832),
       (b'OpenPorchSF', 0.05477668), (b'LotFrontage', 0.05353149),
       (b'Rand_Var', 0.04832145), (b'MasVnrArea', 0.04229633),
       (b'WoodDeckSF', 0.02922249), (b'ExterCondBinary', 0.01939933),
       (b'CentralAirBinary', 0.01918167),
       (b'OverallCondBinary', 0.01714515),
       (b'SaleConditionBinary2', 0.01653209)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [244]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=270,max_depth=5,min_samples_split=100, min_samples_leaf=5, subsample=0.8, random_state=10,max_features=7),
param_grid = param_test5, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch5.fit(x, y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01914, std: 0.00462, params: {'subsample': 0.6},
  mean: -0.01895, std: 0.00452, params: {'subsample': 0.7},
  mean: -0.01905, std: 0.00377, params: {'subsample': 0.75},
  mean: -0.01821, std: 0.00391, params: {'subsample': 0.8},
  mean: -0.01833, std: 0.00433, params: {'subsample': 0.85},
  mean: -0.01902, std: 0.00429, params: {'subsample': 0.9}],
 {'subsample': 0.8},
 -0.018214793335008455)

In [251]:
gbm = GradientBoostingRegressor(learning_rate=0.05, n_estimators=540,max_depth=5,min_samples_split=100, min_samples_leaf=5, subsample=0.8, random_state=10,max_features=7)
gbm.fit(x, y)
gbm.score(x, y)

0.9641196935611226

In [252]:
feature_importance_gbm = list(zip(x.columns, gbm.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.1217761 ), (b'TotalBsmtSF', 0.10177729),
       (b'LotArea', 0.07726483), (b'GarageArea', 0.07507747),
       (b'GarageYrBlt', 0.06767952), (b'YearBuilt', 0.06034558),
       (b'OpenPorchSF', 0.05809184), (b'Rand_Var', 0.05632859),
       (b'LotFrontage', 0.05363976), (b'MasVnrArea', 0.02999669),
       (b'WoodDeckSF', 0.02692323), (b'ExterCondBinary', 0.0212214 ),
       (b'CentralAirBinary', 0.02069086),
       (b'SaleConditionBinary2', 0.01769614),
       (b'OverallCondBinary', 0.01688534)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [191]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

lsvc = Lasso(alpha=0.1).fit(x, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(x)
x.columns[model.get_support()]

Index(['YearBuilt', 'MasVnrArea', 'TotalBsmtSF', 'GrLivArea', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'Rand_Var'],
      dtype='object')

In [26]:
from sklearn.svm import SVR

In [27]:
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(x, y)
clf.score(x, y)

0.8106612936700603